# chunk_predict
1. make blocks
2. download one block and see cell bodies inside the block
3. get segIDs of these by asking neuroglancer with xyz coordinate of the cell body centers

In [1]:
# libraries 1
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd
import csv
from tqdm import tqdm

from cloudvolume import CloudVolume, view
import cc3d
from tifffile.tifffile import imwrite

In [2]:
# libraries 2
# %pwd
sys.path.append(os.path.abspath("../segmentation"))
# to import rootID_lookup and authentication_utils like below

import rootID_lookup as IDlook
import authentication_utils as auth

In [3]:
# cv setting
cv = CloudVolume(auth.get_cv_path('Image')['url'], use_https=True, agglomerate=False)
# loading from US server to use mip0

nuclei_cv = CloudVolume(
    auth.get_cv_path('nuclei_map')['url'],
    progress=False,
    cache=True, # cache to disk to avoid repeated downloads
    use_https=True,
    autocrop=True,
    bounded=False
)

seg = CloudVolume(auth.get_cv_path('FANC_production_segmentation')['url'], use_https=True, agglomerate=False, cache=True)

In [4]:
# make grid to download chunked volumes
[X,Y,Z]=cv.mip_volume_size(0)
# [ 83968 223232   4390]

# cv.mip_voxel_offset(0)
# Vec(0,0,10, dtype=int64)

# we don't need neck conncetive, so lets start below that area y > 75000

step_xy = 128*2**4 # width of each chunk = x or y space between each chunk center in mip0
step_z = 256 # depth of each chunk = z space between each chunk center in mip0

start_x = 128*2**(4-1) # first chunk center
start_y = 128*2**(4-1) + 73728 # step_xy*36=73728
start_z = 256*2**(-1) +10  # 10 is offset

centerX = np.arange(start_x, X, step_xy)
centerY = np.arange(start_y, Y, step_xy)
centerZ = np.arange(start_z, Z, step_z)

# looks okay but there can be only a few space < step/2 at the end of these sequences, causing error when making chunks
if (X - centerX[-1]) < start_x:
    np.put(centerX, -1, X-start_x)
else:
    centerX = np.append(centerX, X-start_x)

if (Y - centerY[-1]) < start_y:
    np.put(centerY, -1, Y-start_y)
else:
    centerY = np.append(centerY, Y-start_y)

if (Z - centerZ[-1]) < start_z:
    np.put(centerZ, -1, Z-start_z)
else:
    centerZ = np.append(centerZ, Z-start_z)

# make nx3 arrays of the chunk center coordinates
chunk_center = np.array(np.meshgrid(centerX, centerY, centerZ), dtype='int64').T.reshape(-1,3)
len(chunk_center)
# 55188
# entire volume was 83160

55188

In [5]:
output=[]

x_thres = 33-10 # 50/(4.3*2^4/45) = 50/1.53
y_thres = 33-10
z_thres = 50-10

connectivity = 26

# thresholding cell body size
def mybbox(img):

    x = np.any(img, axis=(1, 2))
    y = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    xmin, xmax = np.where(x)[0][[0, -1]]
    ymin, ymax = np.where(y)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]

    return xmin, xmax, ymin, ymax, zmin, zmax

In [6]:
# for loop
# for i in tqdm(range(len(chunk_center))):


# here we only use one chunk for testing
# candidate: 39644, 35021(50176, 84992,  1920) has 4 in entire set!!
# in order zxy

# 35021 is 23219 in this setup

nuclei = nuclei_cv.download_point(chunk_center[23219], mip=[68.8,68.8,45.0], size=(128, 128, 256) ) # mip0 and 4 only
# nuclei = nuclei_cv.download_point(chunk_center[i], mip=[68.8,68.8,45.0], size=(128, 128, 256))
# using mio4 to make it faster
# 4.3*(2**4)*128/45=196

In [7]:
# view(nuclei)

In [ ]:
# nuclei_cv.cache.flush()

In [8]:
# cell body detection
# thresholding intensity
mask_temp = nuclei[:,:,:]
mask = np.where(mask_temp > 0.5, 1, 0)  

# print(mask.shape) 
# (128, 128, 256, 1)
mask_s = np.squeeze(mask)
# print(mask_s.shape) 
# (128, 128, 256)

In [9]:
# save images
# volume = mask_s.swapaxes(0, 2).astype('float32')
# imwrite('../Output/mask_s.tif', volume, imagej=True)

# check EM dataset
# tokyo_cv = CloudVolume(
#     auth.get_cv_path('Image_Tokyo')['url'],
#     progress=False,
#     cache=True, # cache to disk to avoid repeated downloads
#     use_https=True
# )

# em = tokyo_cv.download_point(chunk_center[35021], mip=4, size=(128, 128, 256) )
# view(em)

In [ ]:
# nuclei_cv.cache.flush()

In [10]:
# estimate connected components
# # 26-connectivity
# https://en.wikipedia.org/wiki/Pixel_connectivity
# https://github.com/seung-lab/connected-components-3d
cc_out, N = cc3d.connected_components(mask_s, return_N=True, connectivity=connectivity) # free
# print(N)
# 12

In [ ]:
# thresholding cell body size

In [11]:
list=[]
for segid in range(1, N+1):
  extracted_image = cc_out * (cc_out == segid)
  bbox = mybbox(extracted_image)
  list.append(bbox)

print(list)

[(0, 54, 0, 14, 0, 35), (26, 111, 26, 109, 0, 28), (81, 127, 61, 125, 26, 130), (11, 88, 89, 127, 55, 157), (22, 116, 0, 65, 96, 226), (113, 127, 38, 81, 101, 173), (98, 127, 110, 127, 153, 236), (31, 105, 78, 127, 187, 255), (28, 57, 62, 73, 228, 235), (121, 127, 16, 47, 237, 255), (12, 58, 0, 11, 242, 255), (58, 66, 28, 36, 255, 255)]


In [12]:
list2=[]
for segid in range(0, N):
  xwidth = list[segid][1] - list[segid][0]
  ywidth = list[segid][3] - list[segid][2]
  zwidth = list[segid][5] - list[segid][4]
  if xwidth >= x_thres and ywidth >= y_thres and zwidth >= z_thres:
    center = ((list[segid][1] + list[segid][0])/2,
      (list[segid][3] + list[segid][2])/2,
      (list[segid][5] + list[segid][4])/2)
    list2.append(center)
  else:
    pass

print(list2)

[(104.0, 93.0, 78.0), (49.5, 108.0, 106.0), (69.0, 32.5, 161.0), (68.0, 102.5, 221.0)]


In [13]:
# calculate center coordinates of cell bodies

if len(list2): # segIDs_from_pts_cv makes error is there is none in list2
    origin = nuclei.bounds.minpt # 3072,5248,1792
    cell_body_coordinates_mip4 = np.add(np.array(list2), origin)
    cell_body_coordinates = cell_body_coordinates_mip4
    cell_body_coordinates[:,0]  = (cell_body_coordinates_mip4[:,0] * 2**4)
    cell_body_coordinates[:,1]  = (cell_body_coordinates_mip4[:,1] * 2**4)
    cell_body_coordinates = cell_body_coordinates.astype('int64')

    # Lets get IDs using cell_body_coordinates
    cell_body_IDs = IDlook.segIDs_from_pts_cv(pts=cell_body_coordinates, cv=seg) #mip0

    # save
    # type(cell_body_coordinates.shape)
    cell_body_IDs_list = cell_body_IDs.tolist()
    output.append(cell_body_IDs_list)
    # df = pd.DataFrame({'cell_body_coordinates':cell_body_coordinates, 'cell_body_IDs':cell_body_IDs})
    # print(df)
else:
    pass

Segmentation IDs: 100%|██████████| 4/4 [00:00<00:00,  5.94it/s]


In [14]:
nuclei_cv.cache.flush()

In [15]:
# out from for loop
sum = sum(output,[])
output_s = set(sum)
output_str = [str(n) for n in output_s]
output_2D = np.array(output_str ).reshape(len(output_str ),1).tolist()
print(output_2D)

[['648518346490988223'], ['648518346492077650'], ['648518346505454978'], ['648518346490989503']]


In [16]:
with open('../Output/output.csv', 'w') as result:
    writer = csv.writer(result)
    writer.writerows(output_2D)

In [ ]:
# ignore below, just testing a bit for finding neuron partners

In [ ]:
test = seg.download_point(segids=[648518346490989503], mip=0) 


In [ ]:
# test kmeans
# 648518346492077650

mesh = seg.mesh.get(segids=648518346490989503)

In [ ]:
mesh[648518346490989503].vertices

In [ ]:
cell body seg id overlap
cell body and neuron id overlap

In [ ]:
nuclei_cv.cache.flush()

In [ ]:
# nuclei_cv.mip_volume_size(0)
# Vec(86016,225776,4390, dtype=int64)
# entire dataset
# [ 83968 223232   4390]
# why different